In [5]:
#!pip install apache_beam

In [1]:
!echo 'hey how are you?, I am good and you?' >> 'input.txt'

In [2]:
!cat input.txt

hey how are you?, I am good and you?


In [3]:
import re

import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions


input_file = "input.txt"
output_file = "output.txt"

pipeline_options = PipelineOptions()


with beam.Pipeline(options=pipeline_options) as p: 

    # Read the text file[pattern] into a PCollection.
    lines = p | beam.io.ReadFromText(input_file)

    # Count the occurrences of each word.
    counts = (
        lines
        | 'Split' >> (beam.FlatMap(lambda x: re.findall(r'[A-Za-z\']+', x)))
                      # .with_output_types(unicode))
        | 'PairWithOne' >> beam.Map(lambda x: (x, 1))
        | 'GroupAndSum' >> beam.CombinePerKey(sum))

    # Format the counts into a PCollection of strings.
    def format_result(word_count):
        (word, count) = word_count
        return f"{word}: {count}"

    output = counts | 'Format' >> beam.Map(format_result)

    # Write the output using a "Write" transform that has side effects.
    output | beam.io.WriteToText(output_file)

In [4]:
!cat output.txt-00000-of-00001

hey: 1
how: 1
are: 1
you: 2
I: 1
am: 1
good: 1
and: 1
